In [1]:
import os
from ultralytics import YOLO
from ultralytics.nn.modules.conv import *
from ultralytics.nn.modules.block import *
from ultralytics.data.utils import check_det_dataset, img2label_paths
from ultralytics.engine.results import Boxes, Results
from customs.custom_res import CustomedResults
import torch
from PIL import Image
import numpy as np
import cv2

In [2]:
# model = YOLO('trained model/our data/v11s_coco_malaria_PA3.1_7_classes_500epochs.pt')

In [17]:
# path='datasets/Malaria plasmodium RF swap label/test/images/0c0275b6-d874-443e-bbf6-d9518ee0813f_jpg.rf.fc28ccbaf6ad1f71031eb48ffdbc2b4b.jpg'
# path='datasets/Malaria plasmodium RF swap label/test/images/0c7977d3-d985-471d-913f-fc905e3f76e3_jpg.rf.74107807f1582b249470bf35b77f3871.jpg'
# path='datasets/Malaria plasmodium RF swap label/test/images/1afcba30-896d-4594-ad18-5eda52c82753_jpg.rf.826b54999092266bdc4e047f10e76482.jpg'
# path='datasets/pa3_malaria_7_classes/test/images/094.tif'
# results = model(path)
# for result in results:
#     result.save_crop(save_dir=os.path.join('results', os.path.basename(path)), file_name="detection")
# print(type(results[0].boxes))
# for result in results:
#     result.save("annotated_image2.jpg", line_width=4, conf=True)





image 1/1 d:\malaria prediction\parasite_v1\test_code\ultralytics\datasets\pa3_malaria_7_classes\test\images\094.tif: 448x640 10 TAs, 1 S, 202 Healthys, 8 Otherss, 115.1ms
Speed: 5.0ms preprocess, 115.1ms inference, 4.0ms postprocess per image at shape (1, 3, 448, 640)


In [12]:

boxes_data = torch.tensor([[100, 50, 150, 100, 0.9, 0], [200, 150, 300, 250, 0.8, 1]])
orig_shape = (480, 640)  # height, width
boxes = Boxes(boxes_data, orig_shape)
print(boxes.xyxy)
print(boxes.conf)
print(boxes.cls)
print(boxes.xywhn)

tensor([[100.,  50., 150., 100.],
        [200., 150., 300., 250.]])
tensor([0.9000, 0.8000])
tensor([0., 1.])
tensor([[0.1953, 0.1562, 0.0781, 0.1042],
        [0.3906, 0.4167, 0.1562, 0.2083]])


In [4]:
path='datasets/malaria_7_classes_8_folds_temp3/all_train/images/126.jpg'
data_yaml_path = 'datasets/malaria_7_classes_8_folds_temp3/data.yaml'
def draw_gt(path, data_yaml_path):
    image = Image.open(path)
    image_np = np.array(image).astype(np.uint8)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    data_yaml = check_det_dataset(data_yaml_path)
    
    img_height, img_width = image_np.shape[:2]
    label_path = img2label_paths([path.replace("/", "\\")])[0]
    boxes_data = []
    with open(label_path, 'r') as f:
        # Đọc từng dòng
        for line in f:
            # Tách các giá trị trong dòng (theo khoảng trắng)
            values = line.split()
            
            # Lấy class_id, x_center, y_center, width, height
            class_id = int(values[0])
            x_center = float(values[1])
            y_center = float(values[2])
            width = float(values[3])
            height = float(values[4])

            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)
            box_tensor = torch.tensor([x_min, y_min, x_max, y_max, 1.0, class_id])
            boxes_data.append(box_tensor)

    boxes_data = torch.stack(boxes_data)
    # print(boxes_data.ndim)
    # boxes = Boxes(boxes_data, (img_height, img_width))
    results = Results(image_np, path, data_yaml['names'], boxes=boxes_data)

    # for result in results:
    # results.save_crop(save_dir=os.path.join('groundtruth', os.path.basename(path)), file_name="detection")
    print(os.path.join('groundtruth', os.path.basename(path), "annotated_image.jpg"))
    os.makedirs(os.path.join('groundtruth', os.path.basename(path)), exist_ok=True)
    results.save(os.path.join('groundtruth', os.path.basename(path), "annotated_image.jpg"), 
                 conf=False, line_width=3, labels=True)
    
draw_gt(path, data_yaml_path)

groundtruth\126.jpg\annotated_image.jpg


In [2]:
def gen_cls_data(img_folder, tgt_split, data_yaml_path, name_dataset):
    data_yaml = check_det_dataset(data_yaml_path)
    for img_name in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_name)

        image = Image.open(img_path)
        image_np = np.array(image).astype(np.uint8)
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

        img_height, img_width = image_np.shape[:2]
        label_path = img2label_paths([img_path.replace("/", "\\")])[0]
        boxes_data = []
        with open(label_path, 'r') as f:
            # Đọc từng dòng
            for line in f:
                # Tách các giá trị trong dòng (theo khoảng trắng)
                values = line.split()
                
                # Lấy class_id, x_center, y_center, width, height
                class_id = int(values[0])
                x_center = float(values[1])
                y_center = float(values[2])
                width = float(values[3])
                height = float(values[4])

                x_min = int((x_center - width / 2) * img_width)
                y_min = int((y_center - height / 2) * img_height)
                x_max = int((x_center + width / 2) * img_width)
                y_max = int((y_center + height / 2) * img_height)
                box_tensor = torch.tensor([x_min, y_min, x_max, y_max, 1.0, class_id])
                boxes_data.append(box_tensor)

        boxes_data = torch.stack(boxes_data)
        results = CustomedResults(image_np, img_path, data_yaml['names'], boxes=boxes_data)
        results.save_crop_and_box(save_dir=os.path.join('datasets', name_dataset, tgt_split, os.path.basename(img_path.split('.')[0])), file_name="cell")
    return

In [4]:
data_yaml_path = 'datasets/v2_malaria_full_class/data.yaml'
gen_cls_data('datasets/v2_malaria_full_class/train/images', 'train', data_yaml_path, 'v2_malaria_full_class_classification')

In [6]:
gen_cls_data('datasets/v2_malaria_full_class/val/images', 'val', data_yaml_path, 'v2_malaria_full_class_classification')
gen_cls_data('datasets/v2_malaria_full_class/test/images', 'test', data_yaml_path, 'v2_malaria_full_class_classification')

In [4]:
data_yaml_path = 'datasets/malaria_full_class_temp1/data.yaml'
gen_cls_data('datasets/malaria_full_class_temp1/all_train/images', 'train', data_yaml_path, 'malaria_full_class_classification_temp1')

[[1592.0, 1167.0, 1743.0, 1301.0]]
[[745.0, 1286.0, 874.0, 1430.0]]
[[740.0, 75.0, 883.0, 224.0]]
[[1961.0, 964.0, 2110.0, 1120.0]]
[[1632.0, 1681.0, 1756.0, 1813.0]]
[[1498.0, 1551.0, 1640.0, 1671.0]]
[[1590.0, 1302.0, 1708.0, 1420.0]]
[[1774.0, 1489.0, 1920.0, 1636.0]]
[[1662.0, 281.0, 1778.0, 453.0]]
[[1804.0, 132.0, 1951.0, 248.0]]
[[2243.0, 420.0, 2399.0, 523.0]]
[[2327.0, 1601.0, 2506.0, 1756.0]]
[[2405.0, 1750.0, 2538.0, 1912.0]]
[[2514.0, 1694.0, 2658.0, 1863.0]]
[[2419.0, 934.0, 2517.0, 1099.0]]
[[2354.0, 343.0, 2485.0, 453.0]]
[[2486.0, 351.0, 2595.0, 494.0]]
[[2919.0, 324.0, 3065.0, 461.0]]
[[2903.0, 195.0, 3050.0, 324.0]]
[[2812.0, 1407.0, 2959.0, 1529.0]]
[[2868.0, 1687.0, 2981.0, 1802.0]]
[[2910.0, 482.0, 3037.0, 561.0]]
[[18.0, 766.0, 136.0, 907.0]]
[[111.0, 712.0, 223.0, 846.0]]
[[148.0, 1038.0, 274.0, 1195.0]]
[[606.0, 1526.0, 767.0, 1673.0]]
[[651.0, 1328.0, 773.0, 1457.0]]
[[642.0, 1460.0, 796.0, 1553.0]]
[[713.0, 926.0, 821.0, 1065.0]]
[[774.0, 669.0, 917.0, 801.0]]

In [6]:
path='datasets/malaria_full_class_temp1/all_train/images/050.jpg'

def draw_specified_boxes(path, xyxy_list, color=(0, 255, 0), thickness=2):
    # Đọc ảnh
    image = Image.open(path)
    image_np = np.array(image).astype(np.uint8)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    # Vẽ tất cả các bounding box
    for xyxy in xyxy_list:
        x1, y1, x2, y2 = map(int, xyxy)  # Chuyển tọa độ về số nguyên
        cv2.rectangle(image_np, (x1, y1), (x2, y2), color, thickness)

    # Định dạng đường dẫn lưu ảnh
    save_dir = os.path.join('groundtruth', os.path.basename(path))
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, "annotated_image.jpg")

    # Lưu ảnh đã vẽ box
    cv2.imwrite(save_path, image_np)
    print(f"Saved annotated image at: {save_path}")
    
    
xyxy_boxes = [
    [1281.0, 699.0, 1448.0, 859.0],
    [1342.0, 1474.0, 1501.0, 1646.0],
    [1109.0, 691.0, 1248.0, 834.0],
    [1592.0, 1167.0, 1743.0, 1301.0 ],
    [2698.0, 1842.0, 2735.0, 1869.0 ],
]
draw_specified_boxes(path, xyxy_boxes)

Saved annotated image at: groundtruth\050.jpg\annotated_image.jpg
